In [1]:
import seaborn as sns

In [2]:
import pandas as pd

In [6]:
f1_df=pd.read_excel('f1_laps_final_starting_position.xlsx')
f1_df # this is the cleaned data

,Driver,DriverNumber,Team,LapNumber,LapTime,Sector1Time,Sector2Time,Sector3Time,Stint,Compound,...,Sector2Time_s,Sector3Time_s,_Season_str,_Event_raw,_Event_key,_Weather_from_wx,_Winner_from_wx,EventName_Canonical,finishing_position,Starting_Position
0,GAS,10,Red Bull Racing,1,00:01:45.008000,NaN,00:00:24.917000,00:00:38.018000,1,MEDIUM,...,24.917,38.018,2019,Australian Grand Prix,australian grand prix,dry-sunny,BOT,Australian Grand Prix,11.0,17
1,GAS,10,Red Bull Racing,2,00:01:31.585000,00:00:30.704000,00:00:24.245000,00:00:36.636000,1,MEDIUM,...,24.245,36.636,2019,Australian Grand Prix,australian grand prix,dry-sunny,BOT,Australian Grand Prix,11.0,17
2,GAS,10,Red Bull Racing,3,00:01:31.406000,00:00:30.920000,00:00:24.387000,00:00:36.099000,1,MEDIUM,...,24.387,36.099,2019,Australian Grand Prix,australian grand prix,dry-sunny,BOT,Australian Grand Prix,11.0,17
3,GAS,10,Red Bull Racing,4,00:01:32.361000,00:00:32.196000,00:00:24.238000,00:00:35.927000,1,MEDIUM,...,24.238,35.927,2019,Australian Grand Prix,australian grand prix,dry-sunny,BOT,Australian Grand Prix,11.0,17
4,GAS,10,Red Bull Racing,5,00:01:31.171000,00:00:30.366000,00:00:24.301000,00:00:36.504000,1,MEDIUM,...,24.301,36.504,2019,Australian Grand Prix,australian grand prix,dry-sunny,BOT,Australian Grand Prix,11.0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65781,GIO,99,Alfa Romeo Racing,30,00:01:29.775000,00:00:18.146000,00:00:38.496000,00:00:33.133000,2,HARD,...,NaN,NaN,2021,NaN,abu dhabi grand prix,dry,VER,Abu Dhabi Grand Prix,NaN,14
65782,GIO,99,Alfa Romeo Racing,31,00:01:29.505000,00:00:18.093000,00:00:38.532000,00:00:32.880000,2,HARD,...,NaN,NaN,2021,NaN,abu dhabi grand prix,dry,VER,Abu Dhabi Grand Prix,NaN,14
65783,GIO,99,Alfa Romeo Racing,32,00:01:29.596000,00:00:18.098000,00:00:38.526000,00:00:32.972000,2,HARD,...,NaN,NaN,2021,NaN,abu dhabi grand prix,dry,VER,Abu Dhabi Grand Prix,NaN,14
65784,GIO,99,Alfa Romeo Racing,33,00:01:29.442000,00:00:17.989000,00:00:38.608000,00:00:32.845000,2,HARD,...,NaN,NaN,2021,NaN,abu dhabi grand prix,dry,VER,Abu Dhabi Grand Prix,NaN,14


In [7]:
f1_df.dtypes.to_frame("ColumnType")

,ColumnType
Driver,object
DriverNumber,int64
Team,object
LapNumber,int64
LapTime,object
Sector1Time,object
Sector2Time,object
Sector3Time,object
Stint,int64
Compound,object


In [8]:
# Check null counts in the columns of interest
print(f1_df[['LapTime_s', 'Sector2Time_s', 'Sector3Time_s', '_Season_str']].isnull().sum())

LapTime_s        16583
Sector2Time_s    15808
Sector3Time_s    15850
_Season_str          0
dtype: int64


In [10]:
import pandas as pd

# --- Helper: convert a time-like column to seconds (works for datetime.time or strings) ---
def to_seconds(col):
    return pd.to_timedelta(col.astype(str), errors='coerce').dt.total_seconds()

# Columns to inspect/fill
cols_to_check = ['LapTime_s', 'Sector2Time_s', 'Sector3Time_s', '_Season_str']

# 1) See missing values (optionally only for 2021)
mask_2021 = (f1_df.get('Season', pd.Series(dtype='int')).eq(2021)) | (f1_df.get('_Season_str', pd.Series(dtype='object')).astype(str).eq('2021'))
print("Missing (all data):\n", f1_df[cols_to_check].isna().sum())
print("\nMissing (2021 only):\n", f1_df.loc[mask_2021, cols_to_check].isna().sum())

# 2) Build seconds from the raw time columns
f1_df['LapTime_s_from_raw']   = to_seconds(f1_df['LapTime'])
f1_df['Sector1Time_s_from_raw'] = to_seconds(f1_df['Sector1Time'])
f1_df['Sector2Time_s_from_raw'] = to_seconds(f1_df['Sector2Time'])
f1_df['Sector3Time_s_from_raw'] = to_seconds(f1_df['Sector3Time'])

# 3) Fill the numeric seconds columns only where they are missing
f1_df['LapTime_s']     = f1_df['LapTime_s'].fillna(f1_df['LapTime_s_from_raw'])
f1_df['Sector2Time_s'] = f1_df['Sector2Time_s'].fillna(f1_df['Sector2Time_s_from_raw'])
f1_df['Sector3Time_s'] = f1_df['Sector3Time_s'].fillna(f1_df['Sector3Time_s_from_raw'])

# 4) Fill _Season_str from Season (if needed)
if '_Season_str' in f1_df.columns and 'Season' in f1_df.columns:
    f1_df['_Season_str'] = f1_df['_Season_str'].fillna(f1_df['Season'].astype('Int64').astype(str))

# 5) Verify again
print("\nAfter filling (all data):\n", f1_df[cols_to_check].isna().sum())
print("\nAfter filling (2021 only):\n", f1_df.loc[mask_2021, cols_to_check].isna().sum())

# 6) (Optional) cleanup helper columns
f1_df.drop(columns=[
    'LapTime_s_from_raw','Sector1Time_s_from_raw',
    'Sector2Time_s_from_raw','Sector3Time_s_from_raw'
], inplace=True, errors='ignore')

Missing (all data):
 LapTime_s        16583
Sector2Time_s    15808
Sector3Time_s    15850
_Season_str          0
dtype: int64

Missing (2021 only):
 LapTime_s        15856
Sector2Time_s    15708
Sector3Time_s    15710
_Season_str          0
dtype: int64

After filling (all data):
 LapTime_s        1182
Sector2Time_s     135
Sector3Time_s     179
_Season_str         0
dtype: int64

After filling (2021 only):
 LapTime_s        455
Sector2Time_s     35
Sector3Time_s     39
_Season_str        0
dtype: int64


In [11]:
f1_df

,Driver,DriverNumber,Team,LapNumber,LapTime,Sector1Time,Sector2Time,Sector3Time,Stint,Compound,...,Sector2Time_s,Sector3Time_s,_Season_str,_Event_raw,_Event_key,_Weather_from_wx,_Winner_from_wx,EventName_Canonical,finishing_position,Starting_Position
0,GAS,10,Red Bull Racing,1,00:01:45.008000,NaN,00:00:24.917000,00:00:38.018000,1,MEDIUM,...,24.917,38.018,2019,Australian Grand Prix,australian grand prix,dry-sunny,BOT,Australian Grand Prix,11.0,17
1,GAS,10,Red Bull Racing,2,00:01:31.585000,00:00:30.704000,00:00:24.245000,00:00:36.636000,1,MEDIUM,...,24.245,36.636,2019,Australian Grand Prix,australian grand prix,dry-sunny,BOT,Australian Grand Prix,11.0,17
2,GAS,10,Red Bull Racing,3,00:01:31.406000,00:00:30.920000,00:00:24.387000,00:00:36.099000,1,MEDIUM,...,24.387,36.099,2019,Australian Grand Prix,australian grand prix,dry-sunny,BOT,Australian Grand Prix,11.0,17
3,GAS,10,Red Bull Racing,4,00:01:32.361000,00:00:32.196000,00:00:24.238000,00:00:35.927000,1,MEDIUM,...,24.238,35.927,2019,Australian Grand Prix,australian grand prix,dry-sunny,BOT,Australian Grand Prix,11.0,17
4,GAS,10,Red Bull Racing,5,00:01:31.171000,00:00:30.366000,00:00:24.301000,00:00:36.504000,1,MEDIUM,...,24.301,36.504,2019,Australian Grand Prix,australian grand prix,dry-sunny,BOT,Australian Grand Prix,11.0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65781,GIO,99,Alfa Romeo Racing,30,00:01:29.775000,00:00:18.146000,00:00:38.496000,00:00:33.133000,2,HARD,...,38.496,33.133,2021,NaN,abu dhabi grand prix,dry,VER,Abu Dhabi Grand Prix,NaN,14
65782,GIO,99,Alfa Romeo Racing,31,00:01:29.505000,00:00:18.093000,00:00:38.532000,00:00:32.880000,2,HARD,...,38.532,32.880,2021,NaN,abu dhabi grand prix,dry,VER,Abu Dhabi Grand Prix,NaN,14
65783,GIO,99,Alfa Romeo Racing,32,00:01:29.596000,00:00:18.098000,00:00:38.526000,00:00:32.972000,2,HARD,...,38.526,32.972,2021,NaN,abu dhabi grand prix,dry,VER,Abu Dhabi Grand Prix,NaN,14
65784,GIO,99,Alfa Romeo Racing,33,00:01:29.442000,00:00:17.989000,00:00:38.608000,00:00:32.845000,2,HARD,...,38.608,32.845,2021,NaN,abu dhabi grand prix,dry,VER,Abu Dhabi Grand Prix,NaN,14


In [12]:
f1_df.dtypes.to_frame("ColumnType")

,ColumnType
Driver,object
DriverNumber,int64
Team,object
LapNumber,int64
LapTime,object
Sector1Time,object
Sector2Time,object
Sector3Time,object
Stint,int64
Compound,object


In [13]:
# DriverNumber as string
f1_df['DriverNumber'] = f1_df['DriverNumber'].astype(str)

# Season as string
f1_df['Season'] = f1_df['Season'].astype(str)

# Finishing position as nullable integer (keeps blanks for NaN)
f1_df['finishing_position'] = f1_df['finishing_position'].astype('Int64')

In [14]:
f1_df['finishing_position'].isna().sum()

np.int64(4568)

In [15]:
missing_finish = f1_df[f1_df['finishing_position'].isna()]
print(missing_finish[['Driver', 'Season', 'finishing_position']].head(20))  # first 20 for quick look

    Driver Season  finishing_position
458    RIC   2019                <NA>
459    RIC   2019                <NA>
460    RIC   2019                <NA>
461    RIC   2019                <NA>
462    RIC   2019                <NA>
463    RIC   2019                <NA>
464    RIC   2019                <NA>
465    RIC   2019                <NA>
466    RIC   2019                <NA>
467    RIC   2019                <NA>
468    RIC   2019                <NA>
469    RIC   2019                <NA>
470    RIC   2019                <NA>
471    RIC   2019                <NA>
472    RIC   2019                <NA>
473    RIC   2019                <NA>
474    RIC   2019                <NA>
475    RIC   2019                <NA>
476    RIC   2019                <NA>
477    RIC   2019                <NA>


In [16]:
# Create string versions with clear suffix
f1_df['LapNumber_as_str']          = f1_df['LapNumber'].astype(str)
f1_df['Stint_as_str']              = f1_df['Stint'].astype(str)
f1_df['Round_as_str']              = f1_df['Round'].astype(str)
f1_df['finishing_position_as_str'] = f1_df['finishing_position'].astype(str)
f1_df['Starting_Position_as_str']  = f1_df['Starting_Position'].astype(str)

In [17]:
f1_df.dtypes.to_frame("ColumnType")

,ColumnType
Driver,object
DriverNumber,object
Team,object
LapNumber,int64
LapTime,object
Sector1Time,object
Sector2Time,object
Sector3Time,object
Stint,int64
Compound,object


In [18]:
# Save DataFrame to CSV
f1_df.to_csv("f1_laps_times_analysis_cleaned.csv", index=False)